In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm
import re

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 실험군, 대조군 약물 데이터 불러오기 

In [45]:
#실험군 
data = pd.read_csv("./drive/MyDrive/DW_metformin/DQM A Cohort Definitions.csv") 
#대조군 
c= pd.read_csv('./drive/MyDrive/DW_metformin/drug_classification/c.csv')


### 병용 약물 갯수 카운트, 3제요법은 에러  


In [47]:
data['ingredient_count']= [i.count(' / ')+1 for i in data['Name']]
c['ingredient_count']= [i.count(' / ')+1 for i in c['Name']]


In [48]:
#3제요법은 일괄 error
db3= data[data['ingredient_count']==3]
db3['type1']='error'
db3['type2']='error'

DB3= c[c['ingredient_count']==3]
DB3['type1']='error'
DB3['type2']='error'
DB3.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

,Unnamed: 0,Unnamed: 0.1,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2
2212,2212,NaN,37499839,2359358,24 HR empagliflozin 5 MG / linagliptin 2.5 MG ...,Quant Branded Drug,Standard,0,0,Drug,RxNorm,1,3,error,error
2213,2213,NaN,37499838,2359356,24 HR empagliflozin 5 MG / linagliptin 2.5 MG ...,Quant Clinical Drug,Standard,0,0,Drug,RxNorm,1,3,error,error
2214,2214,NaN,37499837,2359353,24 HR empagliflozin 25 MG / linagliptin 5 MG /...,Quant Branded Drug,Standard,0,0,Drug,RxNorm,1,3,error,error
2215,2215,NaN,37499836,2359351,24 HR empagliflozin 25 MG / linagliptin 5 MG /...,Quant Clinical Drug,Standard,0,0,Drug,RxNorm,1,3,error,error
2216,2216,NaN,37499835,2359290,24 HR empagliflozin 12.5 MG / linagliptin 2.5 ...,Quant Branded Drug,Standard,0,0,Drug,RxNorm,1,3,error,error


### 형식에 따른 분류 

In [49]:
#형식에 따라 데이터 군 분류 
### 성분 /성분 제형 : Clinical Dose Group, Branded Drug Form, Clinical Drug Form
### 성분 용량 MG /성분 용량 mg: Branded Drug Comp, Branded Drug, Clinical Drug
### 성분 only : ingredient 

# 성분/ 성분 
condition= (data.Class=='Clinical Drug Form') | (data.Class=='Branded Drug Form') |(data.Class=='Clinical Dose Group')
T1 =data[condition]
condition=  (c.Class=='Clinical Drug Form') | (c.Class=='Branded Drug Form') |(c.Class=='Clinical Dose Group')
C1 = c[condition]

#성분 용량 mg
condition= (data.Class=='Branded Drug Comp') | (data.Class=='Branded Drug') |(data.Class=='Clinical Drug')
T2= data[condition]
condition= (c.Class=='Branded Drug Comp') | (c.Class=='Branded Drug') |(c.Class=='Clinical Drug')
C2= c[condition]

##성분 only
T3= data[data.Class =='Ingredient']
C3 = c[c.Class=='Ingredient']

### 성분/ 성분 형식 분류 

In [20]:
##1제 요법
db1=T1[T1['ingredient_count']==1]
DB1=C1[C1['ingredient_count']==1]
#정규표현식 
reg1= re.compile('(?P<ingredient1>[a-zA-Z]+) [a-zA-Z]+')

matchobj_T= [reg1.search(i)  for i in db1["Name"]]
matchobj_C= [reg1.search(i)  for i in DB1["Name"]]
## 실험군
lists=[]
for v in matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
lists

db1['type1']=lists
db1['type2']=''
### 대조군
lists=[]
for v in matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
lists

DB1['type1']=lists
DB1['type2']=''


db1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2
59,NaN,44187108,OMOP3063831,Metformin Extended Release Oral Tablet [Metfor...,Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
172,NaN,44160816,OMOP3037539,Metformin Extended Release Oral Tablet [Metfor...,Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
202,NaN,44121558,OMOP1116189,Metformin Oral Tablet [Phl-Metformin],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
238,NaN,44108482,OMOP1103113,Metformin Oral Tablet [Bci Metformin],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
239,NaN,44108481,OMOP1103112,Metformin Oral Tablet [Septa-Metformin],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,


In [21]:
#2제 요법
db2=T1[T1['ingredient_count']==2]
DB2=C1[C1['ingredient_count']==2]
#정규표현식 
reg2 = re.compile('(?P<ingredient1>[a-zA-Z]+) / (?P<ingredeint2>[a-zA-Z]+)')
matchobj_T= [reg2.search(i)  for i in db2["Name"]]
matchobj_C= [reg2.search(i)  for i in DB2["Name"]]

#실험군
## type1
lists=[]
for v in matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
db2['type1']=lists
##type 2
## type2
lists=[]
for v in  matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(2).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(2).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
db2['type2']=lists

#대조군 
## type1
lists=[]
for v in matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
DB2['type1']=lists
##type 2
## type2
lists=[]
for v in  matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(2).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(2).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(2).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(2).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(2).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(2).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(2).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(2).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(2).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(2).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
DB2['type2']=lists
DB2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,Unnamed: 0,Unnamed: 0.1,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2
10,10,NaN,45892175,1602112,empagliflozin / linagliptin Oral Tablet [Glyxa...,Branded Drug Form,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
13,13,NaN,45892171,1602108,empagliflozin / linagliptin Oral Tablet,Clinical Drug Form,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
118,118,NaN,44095942,OMOP1090573,empagliflozin / Linagliptin Oral Tablet [Glyxamb],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,2,sglt2,dpp4i
537,537,NaN,43283563,OMOP538392,Linagliptin / Metformin Oral Tablet [Trajentamet],Branded Drug Form,Standard,0,0,Drug,RxNorm Extension,1,2,dpp4i,metformin
1004,1004,NaN,43013893,1368434,alogliptin / pioglitazone Oral Tablet [Oseni],Branded Drug Form,Standard,0,0,Drug,RxNorm,1,2,dpp4i,tzd


In [22]:
## 합치기. 
# 필요한 부분만 추출
db1 = db1[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
db2 = db2[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
DB1 = DB1[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
DB2 = DB2[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
db3 = db3[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
DB3 = DB3[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
sub_total_1 = pd.concat([db1, db2,db3, DB1, DB2,DB3], axis=0)

### 성분 용량 형식: Branded Drug Comp, Branded Drug, Clinical Drug

In [23]:
##1제 요법
db1=T2[T2['ingredient_count']==1]
DB1=C2[C2['ingredient_count']==1]
#정규표현식 
reg1= re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d')

matchobj_T= [reg1.search(i)  for i in db1["Name"]]
matchobj_C= [reg1.search(i)  for i in DB1["Name"]]
## 실험군
lists=[]
for v in matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
lists

db1['type1']=lists
db1['type2']=''
### 대조군
lists=[]
for v in matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
lists

DB1['type1']=lists
DB1['type2']=''


db1.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

,Unnamed: 0,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2
51,NaN,44188426,OMOP3065149,Metformin 1000 MG Extended Release Oral Tablet...,Branded Drug,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
68,NaN,44184571,OMOP3061294,Metformin 850 MG Extended Release Oral Tablet ...,Branded Drug,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
105,NaN,44177151,OMOP3053874,Metformin 850 MG Extended Release Oral Tablet ...,Branded Drug,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
106,NaN,44177150,OMOP3053873,Metformin 500 MG Extended Release Oral Tablet ...,Branded Drug,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,
108,NaN,44177148,OMOP3053871,Metformin 500 MG Extended Release Oral Tablet ...,Branded Drug,Standard,0,0,Drug,RxNorm Extension,1,1,metformin,


In [25]:
#2제 요법
db2=T2[T2['ingredient_count']==2]
DB2=C2[C2['ingredient_count']==2]

#정규표현식 
reg1= re.compile('(?P<ingredient1>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d')
reg2 = re.compile('/ (?P<ingredeint2>[a-zA-Z]+[ ]?[a-zA-Z]+?) \d')

matchobj_T= [reg1.search(i)  for i in db2["Name"]]
matchobj_C= [reg1.search(i)  for i in DB2["Name"]]

## 실험군 type1
lists=[]
for v in matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
db2['type1']=lists

#대조군 
## type1
lists=[]
for v in matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
DB2['type1']=lists
## type 2 정규표현식 
matchobj_T= [reg2.search(i)  for i in db2["Name"]]
matchobj_C= [reg2.search(i)  for i in DB2["Name"]]
## 실험군 type2
lists=[]
for v in  matchobj_T:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
db2['type2']=lists

## 대조군 type2
lists=[]
for v in  matchobj_C:
  if v== None:
    lists.append('error')
  elif  v.group(1).lower().startswith('metformin'):
    lists.append('metformin')
  elif  v.group(1).lower().startswith('gl'):
    lists.append('SU')
  elif v.group(1).lower().endswith('amide'):
    lists.append('SU')
  elif v.group(1).lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.group(1).lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.group(1).lower().endswith('zone'):
    lists.append('tzd')
  elif v.group(1).lower().endswith('glinide'):
    lists.append('gnd')
  elif v.group(1).lower().startswith('mitiglinide'): #mitiglinide calcium hydrate를 제거하기 위함. 
    lists.append('gnd') 
  elif v.group(1).lower().endswith('bose'):
    lists.append('alpha')
  elif v.group(1).lower().startswith('miglitol') :
    lists.append('alpha')
  else:
    lists.append('error')
DB2['type2']=lists
DB2.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:100: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

,Unnamed: 0,Unnamed: 0.1,Id,Code,Name,Class,Standard Concept Caption,RC,DRC,Domain,Vocabulary,Ancestors,ingredient_count,type1,type2
6,6,NaN,45892179,1602120,empagliflozin 25 MG / linagliptin 5 MG Oral Ta...,Branded Drug,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
7,7,NaN,45892178,1602119,empagliflozin 25 MG / linagliptin 5 MG [Glyxambi],Branded Drug Comp,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
8,8,NaN,45892177,1602118,empagliflozin 25 MG / linagliptin 5 MG Oral Ta...,Clinical Drug,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
9,9,NaN,45892176,1602115,empagliflozin 10 MG / linagliptin 5 MG Oral Ta...,Branded Drug,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i
11,11,NaN,45892174,1602111,empagliflozin 10 MG / linagliptin 5 MG [Glyxambi],Branded Drug Comp,Standard,0,0,Drug,RxNorm,1,2,sglt2,dpp4i


In [26]:
## 합치기. 
# 필요한 부분만 추출
db1 = db1[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
db2 = db2[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
DB1 = DB1[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
DB2 = DB2[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
sub_total_2 = pd.concat([db1, db2, DB1, DB2, sub_total_1], axis=0)

### 성분 only: ingredient

In [55]:
#실험군 
name=T3['Name']
lists=[]
for v in name:
  if  v.lower().startswith('gl') or v.lower().startswith('gl'):
    lists.append('SU')
  elif v.lower().endswith('amide') or v.lower().endswith('amide'):
    lists.append('SU')
  elif v.lower().endswith('ozin') or v.lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.lower().endswith('liptin') or v.lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.lower().endswith('zone') or v.lower().endswith('zone'):
    lists.append('tzd')
  elif v.lower().endswith('glinide') or v.lower().endswith('glinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().startswith('mitiglinide') or v.lower().startswith('mitiglinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().endswith('bose') or v.lower().endswith('bose'):
    lists.append('alpha')
  elif v.lower().endswith('miglitol') or v.lower().endswith('miglitol'):
    lists.append('alpha')
  elif v.lower().startswith('metformin'):
    lists.append('metformin')
  else:
    lists.append('error')
lists
T3['type1']=lists
T3['type2']=''
T3['ingredient_count']= 1
# 대조군 
name=C3['Name']
lists=[]
for v in name:
  if  v.lower().startswith('gl') or v.lower().startswith('gl'):
    lists.append('SU')
  elif v.lower().endswith('amide') or v.lower().endswith('amide'):
    lists.append('SU')
  elif v.lower().endswith('ozin') or v.lower().endswith('ozin'):
    lists.append('sglt2')
  elif v.lower().endswith('liptin') or v.lower().endswith('liptin'):
    lists.append('dpp4i')
  elif v.lower().endswith('zone') or v.lower().endswith('zone'):
    lists.append('tzd')
  elif v.lower().endswith('glinide') or v.lower().endswith('glinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().startswith('mitiglinide') or v.lower().startswith('mitiglinide'): #mitiglinide check 
    lists.append('gnd')
  elif v.lower().endswith('bose') or v.lower().endswith('bose'):
    lists.append('alpha')
  elif v.lower().endswith('miglitol') or v.lower().endswith('miglitol'):
    lists.append('alpha')
  elif v.lower().startswith('metformin'):
    lists.append('metformin')
  else:
    lists.append('error')
lists
C3['type1']=lists
C3['type2']=''
C3['ingredient_count']= 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

In [56]:
## 합치기. 
# 필요한 부분만 추출
T3 = T3[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
C3 = C3[['Id', 'Name','Class', 'type1','type2','ingredient_count']]
total= pd.concat([sub_total_2,T3,C3], axis=0)


In [59]:
total.to_csv("./drive/MyDrive/DW_metformin/drug_classification/total.csv")